In [230]:
import pandas as pd
import numpy as np
import unicodedata
df = pd.read_csv("1_Red_vial_nacional_2022-jul2024_0.csv", encoding='utf-8')
print("Primeras filas:")
print(df.head())

df.columns = df.columns.str.strip().str.upper().str.replace(" ", "_")

print("Primeras filas:")
print(df.head())

Primeras filas:
   ID_RVN  ID_DEPARTAMENTO DEPARTAMENTO CODIGO_RUTA  \
0       1               20        PIURA       PE-02   
1       2               20        PIURA       PE-02   
2       3               20        PIURA       PE-02   
3       4               20        PIURA      PE-02A   
4       5               20        PIURA      PE-02A   

                                         TRAYECTORIA  INICIO   FINAL  \
0                     Emp. PE-1N (Dv. Paita) - Paita   0.000   7.918   
1                     Emp. PE-1N (Dv. Paita) - Paita   7.918  40.095   
2                     Emp. PE-1N (Dv. Paita) - Paita  40.095  49.058   
3  Emp. PE-1N J (Dv. Huancabamba) - Buenos Aires ...   0.000  19.720   
4  Emp. PE-1N J (Dv. Huancabamba) - Buenos Aires ...  19.720  27.345   

  CLASIFICACION_EJE JERARQUIA  LONGITUD  NRO_CARRILES  SUPERFICIE  \
0       Transversal        RN     7.918             2           1   
1       Transversal        RN    32.177             2           1   
2       Trans

In [231]:
df = df[[
    "CODIGO_RUTA", "INICIO", "FINAL", "LONGITUD", "NRO_CARRILES",
    "SUPERFICIE_L", "ESTADO_L", "DEPARTAMENTO", "JERARQUIA",
    "CODIGO_CONCESION", "NOMBRE_CONCESION"
]]
df = df.rename(columns={
    "CODIGO_RUTA": "CODRUTA",
    "NRO_CARRILES": "NROCARRIL",
    "CODIGO_CONCESION": "CODCONCES",
    "NOMBRE_CONCESION": "NOMCONCES"
})
print("Primeras filas:")
print(df.head())

Primeras filas:
  CODRUTA  INICIO   FINAL  LONGITUD  NROCARRIL SUPERFICIE_L ESTADO_L  \
0   PE-02   0.000   7.918     7.918          2  Pavimentado    Bueno   
1   PE-02   7.918  40.095    32.177          2  Pavimentado    Bueno   
2   PE-02  40.095  49.058     8.963          2  Pavimentado    Bueno   
3  PE-02A   0.000  19.720    19.720          2  Pavimentado    Bueno   
4  PE-02A  19.720  27.345     7.625          2  Pavimentado    Bueno   

  DEPARTAMENTO JERARQUIA  CODCONCES                           NOMCONCES  
0        PIURA        RN          2       Eje Multimodal Amazonas Norte  
1        PIURA        RN          2       Eje Multimodal Amazonas Norte  
2        PIURA        RN          2       Eje Multimodal Amazonas Norte  
3        PIURA        RN          7  Emp.PE-1B-Buenos Aires - Canchaque  
4        PIURA        RN          7  Emp.PE-1B-Buenos Aires - Canchaque  


In [232]:
print(" Tipos de datos originales:")
print(df.dtypes)
total_original = len(df)
print(f"\n Total de registros originales: {total_original}")

 Tipos de datos originales:
CODRUTA          object
INICIO          float64
FINAL           float64
LONGITUD        float64
NROCARRIL         int64
SUPERFICIE_L     object
ESTADO_L         object
DEPARTAMENTO     object
JERARQUIA        object
CODCONCES         int64
NOMCONCES        object
dtype: object

 Total de registros originales: 20255


In [233]:
eliminados_carril = (df["NROCARRIL"] == -1).sum()
print(f" Registros con NROCARRIL = -1 eliminados: {eliminados_carril}")
df = df[df["NROCARRIL"] != -1]


 Registros con NROCARRIL = -1 eliminados: 3244


In [234]:
# Normalizar texto en columnas clave
for col in ["SUPERFICIE_L", "ESTADO_L", "DEPARTAMENTO", "JERARQUIA", "NOMCONCES"]:
    df[col] = df[col].astype(str).str.strip().str.upper()

# Filtrar solo jerarquía RN y hacer copia explícita
df_rn = df[df["JERARQUIA"] == "RN"].copy()
print(f"Registros con JERARQUIA = 'RN': {len(df_rn)} (de {len(df)})")

# Eliminar duplicados
df_rn = df_rn.drop_duplicates()



Registros con JERARQUIA = 'RN': 17011 (de 17011)


In [235]:
nulos= df.isnull().sum()
porcentaje_antes = (nulos / len(df)) * 100

reporte_antes = pd.DataFrame({
    "NULOS": nulos,
    "PORCENTAJE (%)": porcentaje_antes.round(2)
})

print("NULOS ANTES DE LIMPIAR:\n")
print(reporte_antes)

NULOS ANTES DE LIMPIAR:

              NULOS  PORCENTAJE (%)
CODRUTA           0             0.0
INICIO            0             0.0
FINAL             0             0.0
LONGITUD          0             0.0
NROCARRIL         0             0.0
SUPERFICIE_L      0             0.0
ESTADO_L          0             0.0
DEPARTAMENTO      0             0.0
JERARQUIA         0             0.0
CODCONCES         0             0.0
NOMCONCES         0             0.0


In [236]:
df2 = pd.read_excel("BBDD ONSV - SINIESTROS 2021-2023.xlsx", skiprows=3)

# Mostrar columnas disponibles
print("Columnas encontradas:")
print(df2.columns.tolist())


Columnas encontradas:
['CÓDIGO SINIESTRO', 'FECHA SINIESTRO', 'HORA SINIESTRO', 'CLASE SINIESTRO', 'CANTIDAD DE FALLECIDOS', 'CANTIDAD DE LESIONADOS', 'CANTIDAD DE VEHICULOS DAÑADOS', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'ZONA', 'TIPO DE VÍA', 'RED VIAL', 'COD CARRETERA', 'EXISTE CICLOVÍA', 'COORDENADAS LATITUD', 'COORDENADAS  LONGITUD', 'CONDICIÓN CLIMÁTICA', 'ZONIFICACIÓN', 'CARACTERÍSTICAS DE VÍA', 'PERFIL LONGITUDINAL VÍA', 'SUPERFICIE DE CALZADA', '¿EXISTE SEÑAL VERTICAL?', 'CLASIFICACIÓN DE LA SEÑAL VERTICAL Nº 1', 'CLASIFICACIÓN DE LA SEÑAL VERTICAL Nº 2', '¿EXISTE SEÑAL HORIZONTAL?', 'CAUSA FACTOR PRINCIPAL', 'CAUSA ESPECÍFICA']


In [237]:
df2 = df2[[
    "FECHA SINIESTRO",
    "CANTIDAD DE FALLECIDOS",
    "DEPARTAMENTO",
    "PROVINCIA",
    "DISTRITO",
    "COD CARRETERA",
    "COORDENADAS LATITUD",
    "COORDENADAS  LONGITUD",
    "TIPO DE VÍA",
    "SUPERFICIE DE CALZADA",
    "CONDICIÓN CLIMÁTICA",
    "ZONA",
    "¿EXISTE SEÑAL VERTICAL?",
    "¿EXISTE SEÑAL HORIZONTAL?",
    "CAUSA FACTOR PRINCIPAL",
    "CAUSA ESPECÍFICA"
]]

df2.columns = df2.columns.str.strip().str.upper().str.replace(" ", "_")

In [238]:
# Eliminar filas sin coordenadas (opcional, para mapas)
coordenadas_incompletas = df2["COORDENADAS_LATITUD"].isna() | df2["COORDENADAS__LONGITUD"].isna()
print(f" Registros eliminados por falta de coordenadas: {coordenadas_incompletas.sum()}")
df2 = df2[~coordenadas_incompletas]

 Registros eliminados por falta de coordenadas: 0


In [239]:
# Crear variable binaria de siniestro grave
df2["SINIESTRO_GRAVE"] = df2["CANTIDAD_DE_FALLECIDOS"].apply(lambda x: 1 if x > 0 else 0)


In [240]:
def normalizar_texto(texto):
    if not isinstance(texto, str):
        return texto
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    return texto.strip().upper()

In [241]:
columnas_texto = [
    "DEPARTAMENTO", "PROVINCIA", "DISTRITO", "TIPO_DE_VÍA", "SUPERFICIE_DE_CALZADA",
    "CONDICIÓN_CLIMÁTICA", "ZONA", "¿EXISTE_SEÑAL_VERTICAL?", "¿EXISTE_SEÑAL_HORIZONTAL?",
    "CAUSA_FACTOR_PRINCIPAL", "CAUSA_ESPECÍFICA"
]

In [242]:
for col in columnas_texto:
    df2[col] = df2[col].apply(normalizar_texto)

print("Primeras filas:")
print(df2.head())

Primeras filas:
  FECHA_SINIESTRO  CANTIDAD_DE_FALLECIDOS DEPARTAMENTO PROVINCIA  \
0      01/01/2021                       1         LIMA    HUARAL   
1      01/01/2021                       1         LIMA      LIMA   
2      01/01/2021                       2  LA LIBERTAD      VIRU   
3      01/01/2021                       1  LA LIBERTAD      VIRU   
4      01/01/2021                       1         LIMA      LIMA   

                  DISTRITO   COD_CARRETERA  COORDENADAS_LATITUD  \
0                   HUARAL          LM-671           -11.482879   
1               PACHACAMAC  NO CORRESPONDE           -12.229400   
2                     VIRU         LI-1150            -8.414865   
3                     VIRU           PE-1N            -8.432617   
4  VILLA MARIA DEL TRIUNFO  NO CORRESPONDE           -12.164406   

   COORDENADAS__LONGITUD TIPO_DE_VÍA SUPERFICIE_DE_CALZADA  \
0             -77.255547   CARRETERA                TROCHA   
1             -76.859412     AVENIDA            

In [243]:
# Convertir valores de 'COD_CARRETERA' en nulos
df2["COD_CARRETERA"] = df2["COD_CARRETERA"].replace(
    ["NO CORRESPONDE", "SIN CLASIFICAR"],
    np.nan
)

# Convertir valores de 'CAUSA_FACTOR_PRINCIPAL' en nulos
df2["CAUSA_FACTOR_PRINCIPAL"] = df2["CAUSA_FACTOR_PRINCIPAL"].replace(
    ["NO IDENTIFICA LA CAUSA"],
    np.nan
)

In [244]:
# Reporte inicial de nulos y porcentaje
nulos= df2.isnull().sum()
porcentaje_antes = (nulos / len(df2)) * 100

reporte_antes = pd.DataFrame({
    "NULOS": nulos,
    "PORCENTAJE (%)": porcentaje_antes.round(2)
})

print("NULOS ANTES DE LIMPIAR:\n")
print(reporte_antes)

NULOS ANTES DE LIMPIAR:

                           NULOS  PORCENTAJE (%)
FECHA_SINIESTRO                0            0.00
CANTIDAD_DE_FALLECIDOS         0            0.00
DEPARTAMENTO                   0            0.00
PROVINCIA                      0            0.00
DISTRITO                       0            0.00
COD_CARRETERA               1687           25.10
COORDENADAS_LATITUD            0            0.00
COORDENADAS__LONGITUD          0            0.00
TIPO_DE_VÍA                    0            0.00
SUPERFICIE_DE_CALZADA        313            4.66
CONDICIÓN_CLIMÁTICA          313            4.66
ZONA                          22            0.33
¿EXISTE_SEÑAL_VERTICAL?     5191           77.24
¿EXISTE_SEÑAL_HORIZONTAL?   5191           77.24
CAUSA_FACTOR_PRINCIPAL      3152           46.90
CAUSA_ESPECÍFICA            3218           47.88
SINIESTRO_GRAVE                0            0.00


In [245]:
df2 = df2.drop(columns=['¿EXISTE_SEÑAL_VERTICAL?', '¿EXISTE_SEÑAL_HORIZONTAL?'])

In [246]:
# Reporte final de nulos y porcentaje
nulos= df2.isnull().sum()
porcentaje = (nulos / len(df2)) * 100

reporte_despues = pd.DataFrame({
    "NULOS": nulos,
    "PORCENTAJE (%)": porcentaje.round(2)
})

print("NULOS DESPUES DE LIMPIAR:\n")
print(reporte_despues)

NULOS DESPUES DE LIMPIAR:

                        NULOS  PORCENTAJE (%)
FECHA_SINIESTRO             0            0.00
CANTIDAD_DE_FALLECIDOS      0            0.00
DEPARTAMENTO                0            0.00
PROVINCIA                   0            0.00
DISTRITO                    0            0.00
COD_CARRETERA            1687           25.10
COORDENADAS_LATITUD         0            0.00
COORDENADAS__LONGITUD       0            0.00
TIPO_DE_VÍA                 0            0.00
SUPERFICIE_DE_CALZADA     313            4.66
CONDICIÓN_CLIMÁTICA       313            4.66
ZONA                       22            0.33
CAUSA_FACTOR_PRINCIPAL   3152           46.90
CAUSA_ESPECÍFICA         3218           47.88
SINIESTRO_GRAVE             0            0.00


In [247]:
print(df2.head())

  FECHA_SINIESTRO  CANTIDAD_DE_FALLECIDOS DEPARTAMENTO PROVINCIA  \
0      01/01/2021                       1         LIMA    HUARAL   
1      01/01/2021                       1         LIMA      LIMA   
2      01/01/2021                       2  LA LIBERTAD      VIRU   
3      01/01/2021                       1  LA LIBERTAD      VIRU   
4      01/01/2021                       1         LIMA      LIMA   

                  DISTRITO COD_CARRETERA  COORDENADAS_LATITUD  \
0                   HUARAL        LM-671           -11.482879   
1               PACHACAMAC           NaN           -12.229400   
2                     VIRU       LI-1150            -8.414865   
3                     VIRU         PE-1N            -8.432617   
4  VILLA MARIA DEL TRIUNFO           NaN           -12.164406   

   COORDENADAS__LONGITUD TIPO_DE_VÍA SUPERFICIE_DE_CALZADA  \
0             -77.255547   CARRETERA                TROCHA   
1             -76.859412     AVENIDA             ASFALTADA   
2             

In [248]:
df3 = pd.read_csv(
    "Accidentes de transito en carreteras-2020-2021-Sutran.csv",
    sep=";",
    encoding="latin1"
)
print("Columnas:")
print(df3.columns.tolist())
print("Primeras filas:")
print(df3.head())

Columnas:
['FECHA_CORTE', 'FECHA', 'HORA', 'DEPARTAMENTO', 'CODIGO_VÍA', 'KILOMETRO', 'MODALIDAD', 'FALLECIDOS', 'HERIDOS']
Primeras filas:
   FECHA_CORTE     FECHA   HORA DEPARTAMENTO CODIGO_VÍA KILOMETRO MODALIDAD  \
0     20211222  20200101  05:40         LIMA      PE-1S        24  DESPISTE   
1     20211222  20200101  16:30    CAJAMARCA      PE-3N        74  DESPISTE   
2     20211222  20200101  07:45        PASCO      PE-3N       103  DESPISTE   
3     20211222  20200101  18:30    CAJAMARCA      PE-08       111  DESPISTE   
4     20211222  20200101  18:40         LIMA      PE-1N       174  DESPISTE   

  FALLECIDOS HERIDOS  
0          0       0  
1          0       0  
2          0       1  
3          0       2  
4          0       5  


In [249]:
# 1. Eliminar columna que no se necesita
df3 = df3.drop(columns=["FECHA_CORTE"], errors="ignore")

# 2. Renombrar columnas para consistencia
df3 = df3.rename(columns={
    "CODIGO_VÍA": "CODIGO_VIA",
    "FALLECIDOS": "NUM_FALLECIDOS",
    "HERIDOS": "NUM_HERIDOS"
})

# 3. Reemplazar valores "N.I." o vacíos por NaN
df3 = df3.replace(["N.I.", "N.I", "NI", "", " ", None], np.nan)

In [250]:
# Asegurar que FECHA es string
df3["FECHA"] = df3["FECHA"].astype(str)

# Convertir a datetime y luego al formato deseado
df3["FECHA"] = pd.to_datetime(df3["FECHA"], format="%Y%m%d", errors="coerce")
df3["FECHA"] = df3["FECHA"].dt.strftime("%d/%m/%Y")


In [251]:
df.describe()

,INICIO,FINAL,LONGITUD,NROCARRIL,CODCONCES
count,17011.000000,17011.000000,17011.000000,17011.000000,17011.000000
mean,346.091897,350.172075,4.080081,1.774852,789.510493
std,429.239347,429.168934,6.528386,0.768183,404.000927
min,0.000000,0.048000,0.001000,1.000000,1.000000
25%,57.625000,62.364000,0.500000,1.000000,999.000000
50%,151.179000,155.311000,1.649000,2.000000,999.000000
75%,436.459000,438.303000,4.959000,2.000000,999.000000
max,1793.323000,1804.371000,135.462000,7.000000,999.000000


In [252]:
df3["NUM_FALLECIDOS"] = pd.to_numeric(df3["NUM_FALLECIDOS"], errors="coerce")
df3["NUM_HERIDOS"] = pd.to_numeric(df3["NUM_HERIDOS"], errors="coerce")
df3["KILOMETRO"] = pd.to_numeric(df3["KILOMETRO"], errors="coerce")
print("Primeras filas:")
print(df3.head())

Primeras filas:
        FECHA   HORA DEPARTAMENTO CODIGO_VIA  KILOMETRO MODALIDAD  \
0  01/01/2020  05:40         LIMA      PE-1S       24.0  DESPISTE   
1  01/01/2020  16:30    CAJAMARCA      PE-3N       74.0  DESPISTE   
2  01/01/2020  07:45        PASCO      PE-3N      103.0  DESPISTE   
3  01/01/2020  18:30    CAJAMARCA      PE-08      111.0  DESPISTE   
4  01/01/2020  18:40         LIMA      PE-1N      174.0  DESPISTE   

   NUM_FALLECIDOS  NUM_HERIDOS  
0             0.0          0.0  
1             0.0          0.0  
2             0.0          1.0  
3             0.0          2.0  
4             0.0          5.0  


In [253]:
def normalizar_texto(txt):
    if isinstance(txt, str):
        txt = txt.strip().upper()
        return unicodedata.normalize("NFKD", txt).encode("ascii", "ignore").decode("utf-8")
    return txt

df3["DEPARTAMENTO"] = df3["DEPARTAMENTO"].apply(normalizar_texto)
df3["MODALIDAD"] = df3["MODALIDAD"].apply(normalizar_texto)
df3["CODIGO_VIA"] = df3["CODIGO_VIA"].astype(str).str.strip().str.upper()

In [254]:
# 6. Crear variable binaria de gravedad
df3["SINIESTRO_GRAVE"] = df3["NUM_FALLECIDOS"].apply(lambda x: 1 if pd.notnull(x) and x > 0 else 0)


In [255]:
print("Nulos por columna:")
print(df3.isnull().sum())

Nulos por columna:
FECHA               0
HORA               88
DEPARTAMENTO        7
CODIGO_VIA          0
KILOMETRO          45
MODALIDAD          28
NUM_FALLECIDOS      3
NUM_HERIDOS        10
SINIESTRO_GRAVE     0
dtype: int64


In [256]:
print("\nPrimeras filas del Dataset 3 limpio:")
print(df3.head())


Primeras filas del Dataset 3 limpio:
        FECHA   HORA DEPARTAMENTO CODIGO_VIA  KILOMETRO MODALIDAD  \
0  01/01/2020  05:40         LIMA      PE-1S       24.0  DESPISTE   
1  01/01/2020  16:30    CAJAMARCA      PE-3N       74.0  DESPISTE   
2  01/01/2020  07:45        PASCO      PE-3N      103.0  DESPISTE   
3  01/01/2020  18:30    CAJAMARCA      PE-08      111.0  DESPISTE   
4  01/01/2020  18:40         LIMA      PE-1N      174.0  DESPISTE   

   NUM_FALLECIDOS  NUM_HERIDOS  SINIESTRO_GRAVE  
0             0.0          0.0                0  
1             0.0          0.0                0  
2             0.0          1.0                0  
3             0.0          2.0                0  
4             0.0          5.0                0  


Integración

In [257]:
# Red Vial (df1)
df = df.rename(columns={
    "CODIGO_RUTA": "CODRUTA",
    "DEPARTAMENTO": "DEPARTAMENTO",
    "LONGITUD": "LONGITUD",
    "NRO_CARRILES": "NRO_CARRILES",
    "ESTADO_L": "ESTADO_VIA",
    "SUPERFICIE_L": "SUPERFICIE"
})

# Siniestros ONSV (df2)
df2 = df2.rename(columns={
    "COD_CARRETERA": "CODRUTA"
})

# SUTRAN (df3)
df3 = df3.rename(columns={
    "CODIGO_VIA": "CODRUTA"
})


In [258]:
df1_df2 = df2.merge(df, on="CODRUTA", how="left", suffixes=('', '_RV'))

In [259]:
df1_df3 = df3.merge(df, on="CODRUTA", how="left", suffixes=('', '_RV'))


In [260]:
# Concatenar siniestros de df2 y df3
siniestros = pd.concat([df1_df2, df1_df3], ignore_index=True)

# Asegurar que NUM_FALLECIDOS esté presente y en formato correcto
siniestros["NUM_FALLECIDOS"] = pd.to_numeric(siniestros["NUM_FALLECIDOS"], errors="coerce")
siniestros["SINIESTRO_GRAVE"] = siniestros["SINIESTRO_GRAVE"].fillna(
    siniestros["NUM_FALLECIDOS"].apply(lambda x: 1 if pd.notnull(x) and x > 0 else 0)
)

siniestros_unicos = siniestros.drop_duplicates(
    subset=["CODRUTA", "FECHA", "HORA", "NUM_FALLECIDOS"]
)

resumen = siniestros_unicos.groupby("CODRUTA").agg(
    departamento=("DEPARTAMENTO", "first"),
    num_accidentes=("CODRUTA", "count"),
    num_fallecidos=("NUM_FALLECIDOS", "sum"),
    siniestro_grave=("SINIESTRO_GRAVE", "max"),  # 1 si al menos uno fue grave
    tipo_accidente_mas_frec=("MODALIDAD", lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan),
).reset_index()



In [261]:
dataset_final = df.merge(resumen, on="CODRUTA", how="left")

# Rellenar nulos con valores por defecto
dataset_final["num_accidentes"] = dataset_final["num_accidentes"].fillna(0)
dataset_final["num_fallecidos"] = dataset_final["num_fallecidos"].fillna(0)
dataset_final["siniestro_grave"] = dataset_final["siniestro_grave"].fillna(0).astype(int)


In [269]:
print(dataset_final.head())
dataset_final.describe().T

  CODRUTA  INICIO   FINAL  LONGITUD  NROCARRIL   SUPERFICIE ESTADO_VIA  \
0   PE-02   0.000   7.918     7.918          2  PAVIMENTADO      BUENO   
1   PE-02   7.918  40.095    32.177          2  PAVIMENTADO      BUENO   
2   PE-02  40.095  49.058     8.963          2  PAVIMENTADO      BUENO   
3  PE-02A   0.000  19.720    19.720          2  PAVIMENTADO      BUENO   
4  PE-02A  19.720  27.345     7.625          2  PAVIMENTADO      BUENO   

  DEPARTAMENTO JERARQUIA  CODCONCES                           NOMCONCES  \
0        PIURA        RN          2       EJE MULTIMODAL AMAZONAS NORTE   
1        PIURA        RN          2       EJE MULTIMODAL AMAZONAS NORTE   
2        PIURA        RN          2       EJE MULTIMODAL AMAZONAS NORTE   
3        PIURA        RN          7  EMP.PE-1B-BUENOS AIRES - CANCHAQUE   
4        PIURA        RN          7  EMP.PE-1B-BUENOS AIRES - CANCHAQUE   

  departamento  num_accidentes  num_fallecidos  siniestro_grave  \
0        PIURA            22.0       

,count,mean,std,min,25%,50%,75%,max
INICIO,17011.0,346.091897,429.239347,0.000,57.625,151.179,436.459,1793.323
FINAL,17011.0,350.172075,429.168934,0.048,62.364,155.311,438.303,1804.371
LONGITUD,17011.0,4.080081,6.528386,0.001,0.500,1.649,4.959,135.462
NROCARRIL,17011.0,1.774852,0.768183,1.000,1.000,2.000,2.000,7.000
CODCONCES,17011.0,789.510493,404.000927,1.000,999.000,999.000,999.000,999.000
num_accidentes,17011.0,294.215978,435.365898,0.000,7.000,71.000,472.000,1469.000
num_fallecidos,17011.0,45.794016,75.876224,0.000,2.000,9.000,47.000,272.000
siniestro_grave,17011.0,0.978308,0.145680,0.000,1.000,1.000,1.000,1.000


In [265]:
dataset_final['num_accidentes'] = dataset_final['num_accidentes'].fillna(0)
dataset_final['num_fallecidos'] = dataset_final['num_fallecidos'].fillna(0)
dataset_final['tipo_accidente_mas_frec'] = dataset_final['tipo_accidente_mas_frec'].fillna("NO IDENTIFICADO")
dataset_final["departamento"] = dataset_final["departamento"].fillna(dataset_final["DEPARTAMENTO"])


In [268]:
print(dataset_final.isnull().sum())

CODRUTA                    0
INICIO                     0
FINAL                      0
LONGITUD                   0
NROCARRIL                  0
SUPERFICIE                 0
ESTADO_VIA                 0
DEPARTAMENTO               0
JERARQUIA                  0
CODCONCES                  0
NOMCONCES                  0
departamento               0
num_accidentes             0
num_fallecidos             0
siniestro_grave            0
tipo_accidente_mas_frec    0
dtype: int64


In [267]:
dataset_final.to_csv("dataset_sinaf.csv", index=False, encoding="utf-8")